# bike-reloot data set
## assembling a dat set for a prototype 

This notebook has the purpose of making a data set for the _bike-reloot_ project. 

Steps are the following: 
- scrape will haben bike pictures
- rename them to "stolen" and "not_stolen" with consecutive IDs

---

Inside of a loop through a list of willhaben URLs with different "&pages=1": 

Structure of the "a-tag" I want to scrap, which is one entry of a bike.

```
<a 
    href="/iad/kaufen-und-verkaufen/d/bike-discounter-angebot-kross-trans-4-0-s-28-zoll-0401-578744109/" 
    id="search-result-entry-header-578744109" 
    aria-labelledby="search-result-entry-header-578744109" 
    class="sc-ca51e2d8-0 lkgIJQ sc-deecb898-1 imWndm" 
    data-testid="search-result-entry-header-578744109">
</a>
```

Then retreive the href and save it to a dataframe, for n number of pages which lead to n number of entries (say, 600 entries).


In [1]:
import requests
from bs4 import BeautifulSoup

# DEFINE BASE URL 
# = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-4552&rows=1"
url_fahrräder = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-radsport/fahrraeder-4552"
# and &page=9 &rows=1

# MAKE REQUEST ON BASE URL 
page = requests.get(url_fahrräder)
soup = BeautifulSoup(page.content, "html.parser")

# Define a function to filter elements based on their IDs within the div with ID "skip-to-resultlist"
def find_by_id_custom(tag):
    return (
        tag.name == 'a' and 
        tag.get('id', '').startswith('search-result-entry-header') and 
        tag.has_attr('id')
    )

# FIND ALL a tags within ID
links = soup.find(id="skip-to-resultlist").find_all(find_by_id_custom)
print(len(links))
print(links[0].get('href'))



5
/iad/kaufen-und-verkaufen/d/scotter-769333716/


In [21]:
import pandas as pd

list_id = [ link.get('id') for link in links]
list_href = [ link.get('href') for link in links]

df = pd.DataFrame({"id": list_id,
                   "href": list_href})

# split ID into all parts and extract the unique ID 
df['id_int'] = df['id'].str.split('-',expand=True)[4]
df['links'] = "https://www.willhaben.at"+df['href']
df['links'][1]

'https://www.willhaben.at/iad/kaufen-und-verkaufen/d/hard-enduro-bike-prototyp-scurra-2-769333386/'

# Extracting images of bikes

In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image

base_url = "https://www.willhaben.at"